In [ ]:
import pandas as pd

# 데이터 로드

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

print(train.head())

# EDA - 데이터 로드 & 기본 확인

In [ ]:
print("train shape:", train.shape)
print("test shape:", test.shape)

## 컬럼/타입/결측치 확인

In [ ]:
train.info()

## 타깃 분포 확인 (species)

In [ ]:
train["species"].value_counts().sort_index()

## 기초 통계 (feature 분포)

In [ ]:
train.describe()

## 종별(feature) 평균 비교

In [ ]:
feature_cols = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]

train.groupby("species")[feature_cols].mean()

## 상관관계 확인

In [ ]:
train[feature_cols].corr()

In [ ]:
import matplotlib.pyplot as plt
pd.plotting.scatter_matrix(train[feature_cols], 
figsize=(10,10),                                   #그래프 크기
c=train["species"])                                #종별 색깔 자동 지정
plt.show()

# 전처리

In [ ]:
feature_cols = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]
X = train[feature_cols]                #필요없는 id와 결과에 넣을 species 제거 과정
y = train["species"]

# 베이스라인 모델

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, 
stratify=y,                              #y 비율이 원래 비율이랑 비슷하게 나눠줌
random_state=42)                         #다른 모델에서 사용할때도 동일한 문제지로 비교하기 위한 시드

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = Pipeline([
    ("scaler", StandardScaler()),                      #표준화 해줘야 선형 회귀에서 영향 안받음 0~1로
    ("clf", LogisticRegression(max_iter=2000))         #이터레이션 기본값이 100인데, 거기서 수렴 안되면 모델 성능에 못미치는 값을 줄 수 있어 늘림
    ])

model.fit(X_tr, y_tr)
pred = model.predict(X_va)
print(accuracy_score(y_va, pred))

## 교차 검증과 모델 검증

In [ ]:
from unittest import result
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {
    "LogReg": Pipeline([                             #선형 회귀로 선형으로 무리 자르는 모델
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=2000))
    ]),
    "SVM(RBF)": Pipeline([                           #다른 그룹 개체간 서로 최대거리에 있는 곳에 점을 찍어 잇는 느낌의 모델
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf"))
    ]),
    "KNN": Pipeline([                                #이웃 개수를 보고 많은 쪽으로 변경 그래서 동률이 안나오게 홀수인 5 선택함
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=5))
    ]),
    "RandomForest": RandomForestClassifier(          #성분에 따라 나뉘는 기준 찾는 방법
        n_estimators=300,                            #트리 개수 제한
        random_state=42                              #값 고정에 의미 있어서 숫자는 의미 없어 시드임
    )
}

results = []

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    results.append({
        "model": name,
        "cv_mean": scores.mean(),                    #평균값 이게 높으면 보통 좋음
        "cv_std": scores.std(),                      #평균값 비슷하면 표준편차 낮은게 전체 모델 안정강 더 좋음
        "fold_scores": np.round(scores, 4)           #특정 분류에서 너무 낮은 그룹은 없는지 보기 위함
    })

pd.DataFrame(results).sort_values("cv_mean", ascending=False)

## 회귀분석 튜닝

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Cs = [1, 2, 3, 4, 5, 6, 7]
rows = []

for C in Cs:
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=2000, C=C))              #규제 강도 조절로 작게하면 과적합 줄이고 크게하면 유연해짐
    ])
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    rows.append({"C": C, "mean": scores.mean(), "std": scores.std(), "folds": np.round(scores, 4)})

pd.DataFrame(rows).sort_values("mean", ascending=False)

# 제출파일

In [ ]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, C=3))
])

model.fit(X, y)

test_X = test[feature_cols]
test_pred = model.predict(test_X)

submission = pd.DataFrame({"id": test["id"], "species": test_pred})
submission.to_csv("sample_submission.csv", index=False)

submission.head()